<a href="https://colab.research.google.com/github/Areej73/Prediction-of-Product-Sales/blob/main/Prediction_of_product_Sales_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of product sales

- author: Areej Taha

## project overview

## Load and Inspect Data

## Clean Data

## Exploratory Data Analysis

## Feature Inspection
